# Adler colour measurement

Adler includes a function `col_obs_ref` which calculates the difference between the most recent brightness data point in the observation (obs) filter and a number of previous measurements in the reference (ref) filter.

In [ ]:
from adler.dataclasses.AdlerPlanetoid import AdlerPlanetoid
from adler.dataclasses.AdlerData import AdlerData
from adler.science.PhaseCurve import PhaseCurve
from adler.science.Colour import col_obs_ref
from adler.utilities.plotting_utilities import plot_errorbar
from adler.utilities.science_utilities import apparition_gap_finder, get_df_obs_filt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import astropy.units as u

In [ ]:
# ssObjectId of object to analyse
ssoid = "6098332225018"  # good MBA test object

In [ ]:
# retrieve the object data
fname = "../../notebooks/gen_test_data/adler_demo_testing_database.db"
planetoid = AdlerPlanetoid.construct_from_SQL(ssoid, sql_filename=fname)

In [ ]:
# check orbit parameters
e = planetoid.MPCORB.e
incl = planetoid.MPCORB.incl
q = planetoid.MPCORB.q
a = q / (1.0 - e)
Q = a * (1.0 + e)
print(a, e, incl)

In [ ]:
# plot and fit the phase curves in g and r filters

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

adler_data = AdlerData(ssoid, planetoid.filter_list)

for filt in ["g", "r"]:
    # get observations and define a phase curve model to fit
    sso = planetoid.SSObject_in_filter(filt)
    obs = planetoid.observations_in_filter(filt)

    H = sso.H
    G12 = sso.G12
    pc = PhaseCurve(H=H * u.mag, phase_parameter_1=G12, model_name="HG12_Pen16")

    alpha = np.linspace(0, np.amax(obs.phaseAngle)) * u.deg

    pc_fit = pc.FitModel(
        np.array(getattr(obs, "phaseAngle")) * u.deg,
        np.array(getattr(obs, "reduced_mag")) * u.mag,
    )
    pc = pc.InitModelSbpy(pc_fit)
    red_mag = pc.ReducedMag(alpha)

    adler_data.populate_phase_parameters(filt, **pc.ReturnModelDict())

    # add this phase curve to the figure using the Adler plotting function
    fig = plot_errorbar(planetoid, filt_list=[filt], fig=fig)
    ax1 = fig.axes[0]
    ax1.plot(
        alpha.value,
        pc.ReducedMag(alpha).value,
        label="{}: H={:.2f}, G12={:.2f}".format(filt, pc.H, pc.phase_parameter_1),
    )
    ax1.legend()

ax1.invert_yaxis()

plt.show()

In [ ]:
# inspect the r filter phase curve model
adler_data.get_phase_parameters_in_filter("r", "HG12_Pen16").__dict__

In [ ]:
# inspect the g filter phase curve model
adler_data.get_phase_parameters_in_filter("g", "HG12_Pen16").__dict__

**Determine the apparitions (periods of observability) of the object.**

Get the boundary times for each apparation of the object in the survey using the Adler helper function `apparition_gap_finder`.
In this example we will just look at changes in colour for a single apparition

In [ ]:
# combine all measurements in r and g into one dataframe as apparitions are filter independent
df_obs_all = pd.DataFrame()
for filt in ["r", "g"]:
    obs = planetoid.observations_in_filter(filt)
    _df_obs = pd.DataFrame(obs.__dict__)
    df_obs_all = pd.concat([df_obs_all, _df_obs])
df_obs_all = df_obs_all.sort_values("midPointMjdTai")

# get the boundary times
t_app = apparition_gap_finder(np.array(df_obs_all["midPointMjdTai"]))
print(t_app)

Now we can inpsect how the colour of the object varies (or not) as a function of time. The adler function `col_obs_ref` will compare the latest observation in a given filter with observations in another filter. By setting parameter `N_ref` one can set how many past obsevrations to use when calculating the latest colour.

Here we simulate observations coming night-by-night and the calculation of a g-r colour for the object


In [ ]:
# define colour function parameters

# set number of reference observations to use for colour estimate, there are multiple options
# N_ref = 5
N_ref = 3
# N_ref = 1
# N_ref = None # selecting None uses all previous reference filter measurements

# observation and filter field names
x_plot = "midPointMjdTai"  # time column
y_plot = "reduced_mag"  # magnitude column
yerr_plot = "magErr"  # magnitude uncertainty column
filt_obs = "g"  # observation filter
filt_ref = "r"  # reference filter (we are calculating a filt_obs - filt_ref colour)

# define colour field names
colour = "{}-{}".format(filt_obs, filt_ref)
colErr = "{}-{}Err".format(filt_obs, filt_ref)
delta_t_col = "delta_t_{}".format(colour)
y_ref_col = "{}_{}".format(y_plot, filt_ref)
x1_ref_col = "{}1_{}".format(x_plot, filt_ref)
x2_ref_col = "{}2_{}".format(x_plot, filt_ref)

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

col_dict_list = []
for app_i in range(len(t_app) - 1):
    # consider only one apparition
    if app_i != 3:
        continue

    time_min = t_app[app_i]
    time_max = t_app[app_i + 1]

    _df_obs_all = df_obs_all[
        (df_obs_all["midPointMjdTai"] >= time_min) & (df_obs_all["midPointMjdTai"] < time_max)
    ]
    _time_max = np.amax(_df_obs_all["midPointMjdTai"])

    # get the phase curve model and observations for each filter

    # get the stored AdlerData parameters for the observation filter
    ad_g = adler_data.get_phase_parameters_in_filter(filt_obs, "HG12_Pen16")
    pc_g = PhaseCurve().InitModelDict(ad_g.__dict__)  # make the PhaseCurve object from AdlerData
    # get the phase curve model for the reference filter
    ad_r = adler_data.get_phase_parameters_in_filter(filt_ref, "HG12_Pen16")
    pc_r = PhaseCurve().InitModelDict(ad_r.__dict__)
    # get the observations in both filters
    df_obs = get_df_obs_filt(
        planetoid, filt_obs, x1=time_min, x2=_time_max, col_list=[y_plot, yerr_plot], pc_model=pc_g
    )
    df_obs_ref = get_df_obs_filt(
        planetoid, filt_ref, x1=time_min, x2=_time_max, col_list=[y_plot, yerr_plot], pc_model=pc_r
    )

    ax1.errorbar(df_obs[x_plot], df_obs[y_plot], df_obs[yerr_plot], fmt="o", label=filt_obs)
    ax1.errorbar(df_obs_ref[x_plot], df_obs_ref[y_plot], df_obs_ref[yerr_plot], fmt="o", label=filt_ref)

    # simulate stepping through each new filt_obs observation
    x1 = time_min
    for xi in range(len(df_obs)):
        x2 = df_obs.iloc[xi][x_plot]

        # run the colour finding function here
        col_dict = col_obs_ref(
            planetoid,
            adler_data,
            filt_obs=filt_obs,
            filt_ref=filt_ref,
            N_ref=N_ref,
            x_col=x_plot,
            y_col=y_plot,
            yerr_col=yerr_plot,
            x1=x1,
            x2=x2,
        )
        col_dict_list.append(col_dict)

        # plot some lines to show the colour and mean reference
        ax1.vlines(df_obs.iloc[xi][x_plot], df_obs.iloc[xi][y_plot], col_dict[y_ref_col], color="k", ls=":")
        ax1.hlines(col_dict[y_ref_col], col_dict[x1_ref_col], col_dict[x2_ref_col], color="k", ls="--")

# store running colour parameters as a dataframe
df_col = pd.DataFrame(col_dict_list)
df_col = pd.concat([df_obs, df_col], axis=1)

ax1.set_xlabel(x_plot)
ax1.set_ylabel(y_plot)
ax1.legend()
ax1.invert_yaxis()

plt.show()

In [ ]:
# display the recorded colour parameters
df_col

Now we can plot how the colour changes as a function of time

In [ ]:
# find filt_obs - filt_ref of newest filt_obs observation to the mean of the previous N_ref filt_ref observations
# colour code by time diff between obs and most recent obs_ref

x_plot = "midPointMjdTai"
y_plot = colour
y_plot_err = colErr
c_plot = delta_t_col
df_plot = df_col

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

s1 = ax1.scatter(df_plot[x_plot], df_plot[y_plot], c=df_plot[c_plot], zorder=3)
cbar1 = plt.colorbar(s1)
ax1.errorbar(df_plot[x_plot], df_plot[y_plot], df_plot[yerr_plot], fmt=".", zorder=1)

obs_ref_mean = np.mean(df_plot[y_plot])
obs_ref_std = np.std(df_plot[y_plot])
print("{}-{} mean = {}, std = {}".format(filt_obs, filt_ref, obs_ref_mean, obs_ref_std))

ax1.axhline(obs_ref_mean, c="k")
ax1.axhspan(obs_ref_mean - obs_ref_std, obs_ref_mean + obs_ref_std, zorder=0, color="k", alpha=0.2)

ax1.set_xlabel(x_plot)
ax1.set_ylabel(y_plot)
cbar1.set_label(c_plot)

plt.show()

These colours can then be run through the previously written outlier detection functions.
We have recorded metadata which can help exclude erroneous colour measurements, such as the time difference between the obs and ref measurements.